# Fit & Evaluate Baseline for Nutri-Score
Fit on full training set and evaluate on held out test set

In [ ]:
import pandas as pd
import numpy as np
import time

from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix,
)

In [7]:
# 1. Load data
train = pd.read_csv("../gen/data_def_train_folds.csv")
test  = pd.read_csv("../gen/data_def_test.csv")

print("Train shape:", train.shape)
print("Test shape:", test.shape)

# 2. Create combined column for Phase 2 (intrinsic + extrinsic)
train["labels_1_2_intr_extr"] = (
    train[["labels_1_intrinsic", "labels_2_extrinsic"]]
    .fillna("")
    .agg("|".join, axis=1)
    .str.strip("|")
)

test["labels_1_2_intr_extr"] = (
    test[["labels_1_intrinsic", "labels_2_extrinsic"]]
    .fillna("")
    .agg("|".join, axis=1)
    .str.strip("|")
)

# 3. Target (change only this for NOVA later)
target = "nutriscore"  

# Encode target variable
le = LabelEncoder()
y_train = le.fit_transform(train[target])
y_test = le.transform(test[target])

print("Class mapping:", dict(zip(le.classes_, le.transform(le.classes_))))

# 4. Define phases (name -> column)
phases = {
    "phase1_intrinsic": "labels_1_intrinsic",
    "phase2_intr+extr": "labels_1_2_intr_extr",
    "phase3_all_labels": "labels_string",
}

logreg_params = {
    "solver": "saga",
    "random_state": 42}

# 5. Containers for results
all_metrics = []  
coverage_stats = []  

# 6. Loop over phases – fit/evaluate only (no CV)
for phase_name, label_col in phases.items():
    print("\n" + "="*80)
    print(f"=== {phase_name} – using column '{label_col}' ===")
    print("="*80)

    # 6.1 Prepare label features for this phase
    train[label_col] = train[label_col].fillna("")
    test[label_col]  = test[label_col].fillna("")

    train_labels = train[label_col].apply(lambda x: x.split("|") if x != "" else [])
    test_labels  = test[label_col].apply(lambda x: x.split("|") if x != "" else [])

    mlb = MultiLabelBinarizer(sparse_output=True)
    X_train_labels = mlb.fit_transform(train_labels)
    X_test_labels  = mlb.transform(test_labels)

    print(f"Encoded label matrix shape (train): {X_train_labels.shape}")
    print(f"Encoded label matrix shape (test):  {X_test_labels.shape}")

    # PhaseCoverage (at least 1 label in this phase)
    train_nonempty_mask = np.array(X_train_labels.sum(axis=1)).ravel() > 0
    test_nonempty_mask  = np.array(X_test_labels.sum(axis=1)).ravel() > 0

    # Basic counts for coverage
    train_total = len(y_train)
    test_total  = len(y_test)
    core_mask   = test["core_slice"].astype(bool).values
    core_total  = int(core_mask.sum())

    train_cov_n = int(train_nonempty_mask.sum())
    test_cov_n  = int(test_nonempty_mask.sum())
    core_cov_n  = int((test_nonempty_mask & core_mask).sum())

    # Store coverage stats
    coverage_stats.append({
        "Phase": phase_name,
        "Dataset": "Train",
        "n_total": train_total,
        "n_with_labels": train_cov_n,
        "frac_with_labels": train_cov_n / train_total if train_total > 0 else np.nan,
    })
    coverage_stats.append({
        "Phase": phase_name,
        "Dataset": "Test",
        "n_total": test_total,
        "n_with_labels": test_cov_n,
        "frac_with_labels": test_cov_n / test_total if test_total > 0 else np.nan,
    })
    coverage_stats.append({
        "Phase": phase_name,
        "Dataset": "Test-Core",
        "n_total": core_total,
        "n_with_labels": core_cov_n,
        "frac_with_labels": core_cov_n / core_total if core_total > 0 else np.nan,
    })

    print(
        f"Coverage – {phase_name}:\n"
        f"  Train:     {train_cov_n}/{train_total} "
        f"({train_cov_n/train_total:.3%} with ≥1 label in this phase)\n"
        f"  Test:      {test_cov_n}/{test_total} "
        f"({test_cov_n/test_total:.3%} with ≥1 label in this phase)\n"
        f"  Test-Core: {core_cov_n}/{core_total} "
        f"({core_cov_n/core_total:.3%} with ≥1 label in this phase)"
    )

    # 6.2 Fit on full training set
    logreg = LogisticRegression(**logreg_params)

    fit_start = time.perf_counter()
    logreg.fit(X_train_labels, y_train)
    fit_end = time.perf_counter()
    fit_time = fit_end - fit_start

    print(f"Fitted LogisticRegression for {phase_name} in {fit_time:.2f} s")

    # 6.3 Predictions: Train–Overall, Test–Overall, Test–Core
    # Train–Overall
    pred_train_start = time.perf_counter()
    y_pred_train = logreg.predict(X_train_labels)
    pred_train_end = time.perf_counter()
    pred_train_time = pred_train_end - pred_train_start

    # Test–Overall
    pred_test_start = time.perf_counter()
    y_pred_test = logreg.predict(X_test_labels)
    pred_test_end = time.perf_counter()
    pred_test_time = pred_test_end - pred_test_start

    # Test–Core: subset where core_slice == True
    if core_mask.sum() == 0:
        print("Warning: no rows with core_slice == True in test set.")
        y_test_core = np.array([])
        y_pred_test_core = np.array([])
    else:
        y_test_core = y_test[core_mask]
        y_pred_test_core = y_pred_test[core_mask]

    # PhaseCoverage subsets (at least 1 label in this phase)
    y_train_cov = y_train[train_nonempty_mask]
    y_pred_train_cov = y_pred_train[train_nonempty_mask]

    y_test_cov = y_test[test_nonempty_mask]
    y_pred_test_cov = y_pred_test[test_nonempty_mask]

    # 6.4 Attach predictions back to train/test DataFrames
    # numeric predictions (encoded)
    train[f"{phase_name}_pred_enc"] = y_pred_train
    test[f"{phase_name}_pred_enc"] = y_pred_test

    # decoded predictions (original class labels, e.g. 'A','B','C','D','E')
    train[f"{phase_name}_pred"] = le.inverse_transform(y_pred_train)
    test[f"{phase_name}_pred"] = le.inverse_transform(y_pred_test)

    # 6.5 Compute metrics for each dataset view
    def compute_metrics(y_true, y_pred, phase, dataset_name):
        """Return a dict of metrics for convenience."""
        if len(y_true) == 0:
            # In case a slice ends up empty, return NaNs
            return {
                "Phase": phase,
                "Dataset": dataset_name,
                "n_samples": 0,
                "Accuracy": np.nan,
                "Balanced Accuracy": np.nan,
                "Precision (Macro)": np.nan,
                "Recall (Macro)": np.nan,
                "F1 (Macro)": np.nan,
                "F1 (Micro)": np.nan,
            }

        acc = accuracy_score(y_true, y_pred)
        bal_acc = balanced_accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred, average="macro", zero_division=0)
        rec = recall_score(y_true, y_pred, average="macro", zero_division=0)
        f1_mac = f1_score(y_true, y_pred, average="macro", zero_division=0)
        f1_mic = f1_score(y_true, y_pred, average="micro", zero_division=0)

        return {
            "Phase": phase,
            "Dataset": dataset_name,
            "n_samples": len(y_true),
            "Accuracy": acc,
            "Balanced Accuracy": bal_acc,
            "Precision (Macro)": prec,
            "Recall (Macro)": rec,
            "F1 (Macro)": f1_mac,
            "F1 (Micro)": f1_mic,
        }

    # Train Metrics
    metrics_train = compute_metrics(y_train, y_pred_train, phase_name, "Train-Overall")
    all_metrics.append(metrics_train)

    metrics_train_cov = compute_metrics(y_train_cov, y_pred_train_cov, phase_name, "Train-PhaseCoverage")
    all_metrics.append(metrics_train_cov)

    # Test Metrics
    metrics_test = compute_metrics(y_test, y_pred_test, phase_name, "Test-Overall")
    all_metrics.append(metrics_test)

    metrics_test_cov  = compute_metrics(y_test_cov,  y_pred_test_cov,  phase_name, "Test-PhaseCoverage")
    all_metrics.append(metrics_test_cov)

    metrics_core = compute_metrics(y_test_core, y_pred_test_core, phase_name, "Test-Core")
    all_metrics.append(metrics_core)

    # Print summary for quick inspection
    print("\nMetrics –", phase_name)
    for m in [metrics_train, metrics_test, metrics_core, metrics_train_cov, metrics_test_cov]:
        print(
            f"{m['Dataset']}: "
            f"n={m['n_samples']}, "
            f"Acc={m['Accuracy']:.4f} "
            f"(Bal={m['Balanced Accuracy']:.4f}, "
            f"F1_macro={m['F1 (Macro)']:.4f})"
        )

# 7. Combine and inspect metrics across phases & datasets
metrics_df = pd.DataFrame(all_metrics)
coverage_df = pd.DataFrame(coverage_stats)

print("\n=== All Metrics (Train-Overall, Test-Overall, Test-Core, PhaseCoverage) ===")
print(metrics_df)

print("\n=== Coverage stats per phase & dataset ===")
print(coverage_df)

# 8. Save outputs
metrics_df.to_csv("../results/logreg_nutri_eval_metrics_train_test_core.csv", sep=";", index=False)
coverage_df.to_csv("../results/logreg_nutri_phase_coverage_stats.csv", sep=";", index=False)
train.to_csv("../results/data_def_train_with_baseline_nutri_preds.csv", sep=";", index=False)
test.to_csv("../results/data_def_test_with_baseline_nutri_preds.csv", sep=";", index=False)

print("\nFiles written:")
print(" - logreg_nutri_eval_metrics_train_test_core.csv")
print(" - logreg_nutri_phase_coverage_stats.csv")
print(" - data_def_train_with_baseline_nutri_preds.csv")
print(" - data_def_test_with_baseline_nutri_preds.csv")

/var/folders/p2/b69xrmz146zc98k0yh6ylk1w0000gp/T/ipykernel_14850/3248772743.py:20: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("data_def_train_folds.csv")
/var/folders/p2/b69xrmz146zc98k0yh6ylk1w0000gp/T/ipykernel_14850/3248772743.py:21: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  test  = pd.read_csv("data_def_test.csv")


Train shape: (206068, 24)
Test shape: (88315, 23)
Class mapping: {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4}

=== phase1_intrinsic – using column 'labels_1_intrinsic' ===


/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['ARTIFICIAL_FLAVOR', 'ENRICHED_WITH_OMEGA3', 'HIGH_IN_CARBOHYDRATES', 'HIGH_IN_OMEGA3_AND_OMEGA6', 'HIGH_IN_PROTEIN_AND_FIBRE', 'HIGH_IN_VITAMIN_K', 'KETOGENIC', 'LOW_FODMAP', 'MINIMALLY_PROCESSED', 'NATURAL_SUGARS_FROM_OATS', 'NATURAL_SUGARS_UNALTERED', 'NOT_FROZEN', 'NO_ADDED_ARTIFICIAL_SWEETENERS', 'NO_ADDITIVES_PRESERVATIVES_COLORANTS', 'NO_ANHYDRIDES', 'NO_AROMA_COLORANT_PRESERVATIVE', 'NO_ARTIFICIAL_DYES', 'NO_ARTIFICIAL_E_NUMBERS', 'NO_ARTIFICIAL_PRESERVATIVES_FLAVOURS_DYES', 'NO_CARRAGEENAN', 'NO_CHEMICAL_INGREDIENTS', 'NO_CHEMICAL_TREATMENT', 'NO_CLEANING_AGENTS', 'NO_COLORANT_AROMA_PRESERVATIVE', 'NO_DYES_PRESERVATIVES_HYDROGENATED_FATS', 'NO_EXTRACTS', 'NO_FRUIT', 'NO_GARLIC_ONION', 'NO_GLUTEN_COLORANT_PRESERVATIVE', 'NO_HIGH_PRESSURE_PROCESSING', 'NO_HONEY', 'NO_LARD', 'NO_METAL_MOLD', 'NO_NUT_TRACES', 'NO_PALM_OIL_DERIVATIVES', 'NO_PHOSPHORIC_ACID', 'NO_POTATO_F

Encoded label matrix shape (train): (206068, 359)
Encoded label matrix shape (test):  (88315, 359)
Coverage – phase1_intrinsic:
  Train:     80574/206068 (39.101% with ≥1 label in this phase)
  Test:      34289/88315 (38.826% with ≥1 label in this phase)
  Test-Core: 4875/4876 (99.979% with ≥1 label in this phase)
Fitted LogisticRegression for phase1_intrinsic in 5.74 s

Metrics – phase1_intrinsic
Train-Overall: n=206068, Acc=0.2992 (Bal=0.2521, F1_macro=0.2033)
Test-Overall: n=88315, Acc=0.2979 (Bal=0.2513, F1_macro=0.2018)
Test-Core: n=4876, Acc=0.3987 (Bal=0.3702, F1_macro=0.3512)
Train-PhaseCoverage: n=80574, Acc=0.3578 (Bal=0.3265, F1_macro=0.3095)
Test-PhaseCoverage: n=34289, Acc=0.3541 (Bal=0.3240, F1_macro=0.3058)

=== phase2_intr+extr – using column 'labels_1_2_intr_extr' ===


/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['ARTIFICIAL_FLAVOR', 'CERT_BIODYNAMIC', 'COASTAL_FISHING', 'DLG_BRONZE_MEDAL', 'ENRICHED_WITH_OMEGA3', 'EU_ORGANIC_CERT_FB', 'EU_ORGANIC_CERT_MY', 'GLUTEN_FREE_VEGAN', 'GLUTEN_LACTOSE_SULFITE_FREE', 'HAZELNUT_FREE', 'HIGH_IN_CARBOHYDRATES', 'HIGH_IN_OMEGA3_AND_OMEGA6', 'HIGH_IN_PROTEIN_AND_FIBRE', 'HIGH_IN_VITAMIN_K', 'INVALID_OR_ERROR', 'ISO_50001', 'KETOGENIC', 'LACTOSE_DAIRY_FREE', 'LOW_FODMAP', 'MAY_CONTAIN_SULPHITES', 'MILK_DERIVATIVE_FREE', 'MINIMALLY_PROCESSED', 'NATURAL_SUGARS_FROM_OATS', 'NATURAL_SUGARS_UNALTERED', 'NOT_FROZEN', 'NO_ADDED_ARTIFICIAL_SWEETENERS', 'NO_ADDITIVES_PRESERVATIVES_COLORANTS', 'NO_ANHYDRIDES', 'NO_ANIMAL_INGREDIENTS', 'NO_AROMA_COLORANT_PRESERVATIVE', 'NO_ARTIFICIAL_DYES', 'NO_ARTIFICIAL_E_NUMBERS', 'NO_ARTIFICIAL_PRESERVATIVES_FLAVOURS_DYES', 'NO_CARRAGEENAN', 'NO_CHEMICAL_INGREDIENTS', 'NO_CHEMICAL_TREATMENT', 'NO_CLEANING_AGENTS', 'NO_COL

Encoded label matrix shape (train): (206068, 808)
Encoded label matrix shape (test):  (88315, 808)
Coverage – phase2_intr+extr:
  Train:     185867/206068 (90.197% with ≥1 label in this phase)
  Test:      79571/88315 (90.099% with ≥1 label in this phase)
  Test-Core: 4876/4876 (100.000% with ≥1 label in this phase)
Fitted LogisticRegression for phase2_intr+extr in 9.76 s

Metrics – phase2_intr+extr
Train-Overall: n=206068, Acc=0.3692 (Bal=0.3379, F1_macro=0.3125)
Test-Overall: n=88315, Acc=0.3641 (Bal=0.3335, F1_macro=0.3067)
Test-Core: n=4876, Acc=0.4237 (Bal=0.3981, F1_macro=0.3838)
Train-PhaseCoverage: n=185867, Acc=0.3785 (Bal=0.3496, F1_macro=0.3260)
Test-PhaseCoverage: n=79571, Acc=0.3728 (Bal=0.3446, F1_macro=0.3196)

=== phase3_all_labels – using column 'labels_string' ===


/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['ARTIFICIAL_FLAVOR', 'CARREFOUR_CLASSIC', 'CERT_BIODYNAMIC', 'COASTAL_FISHING', 'DLG_BRONZE_MEDAL', 'ENRICHED_WITH_OMEGA3', 'EU_ORGANIC_CERT_FB', 'EU_ORGANIC_CERT_MY', 'GLUTEN_FREE_VEGAN', 'GLUTEN_LACTOSE_SULFITE_FREE', 'GOLD_MEDAL_GENERIC', 'GOLD_MEDAL_MILK_GRADE_A', 'HAZELNUT_FREE', 'HIGH_IN_CARBOHYDRATES', 'HIGH_IN_OMEGA3_AND_OMEGA6', 'HIGH_IN_PROTEIN_AND_FIBRE', 'HIGH_IN_VITAMIN_K', 'INVALID_OR_ERROR', 'ISO_50001', 'KETOGENIC', 'LACTOSE_DAIRY_FREE', 'LOW_FODMAP', 'MAY_CONTAIN_SULPHITES', 'MILK_DERIVATIVE_FREE', 'MINIMALLY_PROCESSED', 'NATURAL_SUGARS_FROM_OATS', 'NATURAL_SUGARS_UNALTERED', 'NOT_FROZEN', 'NO_ADDED_ARTIFICIAL_SWEETENERS', 'NO_ADDITIVES_PRESERVATIVES_COLORANTS', 'NO_ANHYDRIDES', 'NO_ANIMAL_INGREDIENTS', 'NO_AROMA_COLORANT_PRESERVATIVE', 'NO_ARTIFICIAL_DYES', 'NO_ARTIFICIAL_E_NUMBERS', 'NO_ARTIFICIAL_PRESERVATIVES_FLAVOURS_DYES', 'NO_CARRAGEENAN', 'NO_CHEMICA

Encoded label matrix shape (train): (206068, 870)
Encoded label matrix shape (test):  (88315, 870)
Coverage – phase3_all_labels:
  Train:     206068/206068 (100.000% with ≥1 label in this phase)
  Test:      88302/88315 (99.985% with ≥1 label in this phase)
  Test-Core: 4876/4876 (100.000% with ≥1 label in this phase)
Fitted LogisticRegression for phase3_all_labels in 10.66 s

Metrics – phase3_all_labels
Train-Overall: n=206068, Acc=0.3760 (Bal=0.3475, F1_macro=0.3230)
Test-Overall: n=88315, Acc=0.3698 (Bal=0.3417, F1_macro=0.3157)
Test-Core: n=4876, Acc=0.4346 (Bal=0.4097, F1_macro=0.4032)
Train-PhaseCoverage: n=206068, Acc=0.3760 (Bal=0.3475, F1_macro=0.3230)
Test-PhaseCoverage: n=88302, Acc=0.3699 (Bal=0.3417, F1_macro=0.3158)

=== All Metrics (Train-Overall, Test-Overall, Test-Core, PhaseCoverage) ===
                Phase              Dataset  n_samples  Accuracy  \
0    phase1_intrinsic        Train-Overall     206068  0.299207   
1    phase1_intrinsic  Train-PhaseCoverage      8